In [2]:
import numpy as np
import tensorflow as tf
from IPython.display import HTML
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from node.core import get_node_function
from node.solvers import RK4Solver
from node.utils.trajectory import tracer
from node.hopfield import hopfield, identity


# for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


DTYPE = 'float32'
tf.keras.backend.set_floatx(DTYPE)


def process(X, y):
    X = X / 255.
    X = np.reshape(X, [-1, 28 * 28])
    y = np.eye(10)[y]
    return X.astype(DTYPE), y.astype(DTYPE)


mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = process(x_train, y_train)
x_test, y_test = process(x_test, y_test)

scalar = StandardScaler()
scalar.fit(x_train)
x_train = scalar.transform(x_train)
x_test = scalar.transform(x_test)

In [3]:
base_model = tf.keras.Sequential([
    tf.keras.layers.Input([28 * 28]),
    tf.keras.layers.Dense(64, use_bias=False),  # down-sampling
    tf.keras.layers.Dense(10, activation='softmax')
])

base_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Nadam(1e-3),
    metrics=['accuracy'])

base_model.fit(x_train, y_train, epochs=10, batch_size=128)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 25us/sample - loss: 0.4483 - accuracy: 0.8815
Epoch 2/10
60000/60000 [==============================] - 1s 20us/sample - loss: 0.3178 - accuracy: 0.9155
Epoch 3/10
60000/60000 [==============================] - 1s 18us/sample - loss: 0.2941 - accuracy: 0.9198
Epoch 4/10
60000/60000 [==============================] - 1s 18us/sample - loss: 0.2852 - accuracy: 0.9218
Epoch 5/10
60000/60000 [==============================] - 1s 15us/sample - loss: 0.2772 - accuracy: 0.9245
Epoch 6/10
60000/60000 [==============================] - 1s 15us/sample - loss: 0.2729 - accuracy: 0.9259
Epoch 7/10
60000/60000 [==============================] - 1s 15us/sample - loss: 0.2689 - accuracy: 0.9267
Epoch 8/10
60000/60000 [==============================] - 1s 18us/sample - loss: 0.2650 - accuracy: 0.9266
Epoch 9/10
60000/60000 [==============================] - 1s 15us/sample - loss: 0.2640 - accuracy: 0.9284
Epoch 10/10
60

In [4]:
class NonHopfieldLayer(tf.keras.layers.Layer):

    def __init__(self, units, dt, num_grids, **kwargs):
        super().__init__(**kwargs)
        self.dt = dt
        self.num_grids = num_grids

        t0 = tf.constant(0., dtype=DTYPE)
        self.tN = t0 + num_grids * dt

        self._model = tf.keras.Sequential([
            tf.keras.layers.Dense(1024, activation='relu', dtype=DTYPE),
            tf.keras.layers.Dense(units, dtype=DTYPE),
        ])
        self._model.build([None, units])
        self._pvf = lambda _, x: self._model(x)

        self._node_fn = get_node_function(RK4Solver(self.dt, dtype=DTYPE),
                                          tf.constant(0., dtype=DTYPE),
                                          self._pvf)

    def call(self, x):
        y = self._node_fn(self.tN, x)
        return y

    def get_config(self):
        return super().get_config().copy()
    

non_hopfield_model = tf.keras.Sequential([
    tf.keras.layers.Input([28 * 28]),
    tf.keras.layers.Dense(64, use_bias=False),  # down-sampling
    NonHopfieldLayer(64, dt=1e-1, num_grids=10),
    tf.keras.layers.Dense(10, activation='softmax')
])

non_hopfield_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Nadam(1e-3),
    metrics=['accuracy'])

non_hopfield_model.fit(x_train, y_train, epochs=10, batch_size=128)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 41s 691us/sample - loss: 0.3143 - accuracy: 0.9143
Epoch 2/10
60000/60000 [==============================] - 42s 701us/sample - loss: 0.1305 - accuracy: 0.9620
Epoch 3/10
60000/60000 [==============================] - 46s 771us/sample - loss: 0.0865 - accuracy: 0.9738
Epoch 4/10
60000/60000 [==============================] - 45s 756us/sample - loss: 0.0654 - accuracy: 0.9807
Epoch 5/10
60000/60000 [==============================] - 49s 822us/sample - loss: 0.0556 - accuracy: 0.9831
Epoch 6/10
60000/60000 [==============================] - 54s 906us/sample - loss: 0.0461 - accuracy: 0.9862
Epoch 7/10
60000/60000 [==============================] - 54s 908us/sample - loss: 0.0354 - accuracy: 0.9892
Epoch 8/10
60000/60000 [==============================] - 55s 924us/sample - loss: 0.0336 - accuracy: 0.9899
Epoch 9/10
60000/60000 [==============================] - 50s 828us/sample - loss: 0.0325 - accuracy: 0.9

In [21]:
longer_trajectory_non_hopfield_model = tf.keras.Sequential([
    tf.keras.layers.Input([28 * 28]),
    tf.keras.layers.Dense(64, use_bias=False),  # down-sampling
    NonHopfieldLayer(64, dt=1e-1, num_grids=50),
    tf.keras.layers.Dense(10, activation='softmax')
])

longer_trajectory_non_hopfield_model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'])

longer_trajectory_non_hopfield_model.set_weights(non_hopfield_model.get_weights())
longer_trajectory_non_hopfield_model.evaluate(x_train, y_train, batch_size=128)

60000/60000 [==============================] - 88s 1ms/sample - loss: 0.2954 - accuracy: 0.9311


[0.29542918457984924, 0.9310667]

In [16]:
class HopfieldLayer(tf.keras.layers.Layer):

    def __init__(self, units, dt, num_grids, **kwargs):
        super().__init__(**kwargs)
        self.dt = dt
        self.num_grids = num_grids

        t0 = tf.constant(0., dtype=DTYPE)
        self.tN = t0 + num_grids * dt

        self._model = tf.keras.Sequential([
            tf.keras.layers.Dense(1024, activation='relu', dtype=DTYPE),
            tf.keras.layers.Dense(units, dtype=DTYPE),
        ])
        self._model.build([None, units])
        self._pvf = hopfield(identity, lambda x: tf.reduce_sum(tf.square(self._model(x))))

        self._node_fn = get_node_function(RK4Solver(self.dt, dtype=DTYPE),
                                          tf.constant(0., dtype=DTYPE),
                                          self._pvf)

    def call(self, x):
        y = self._node_fn(self.tN, x)
        return y

    def get_config(self):
        return super().get_config().copy()


hopfield_model = tf.keras.Sequential([
    tf.keras.layers.Input([28 * 28]),
    tf.keras.layers.Dense(64, use_bias=False),  # down-sampling
    HopfieldLayer(64, dt=1e-1, num_grids=10),
    tf.keras.layers.Dense(10, activation='softmax')
])

hopfield_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Nadam(1e-3, epsilon=1e-2),
    metrics=['accuracy'])

hopfield_model.fit(x_train, y_train, epochs=10, batch_size=128)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 173s 3ms/sample - loss: 0.4003 - accuracy: 0.8871
Epoch 2/10
60000/60000 [==============================] - 174s 3ms/sample - loss: 0.1850 - accuracy: 0.9470
Epoch 3/10
60000/60000 [==============================] - 173s 3ms/sample - loss: 0.1312 - accuracy: 0.9611
Epoch 4/10
60000/60000 [==============================] - 173s 3ms/sample - loss: 0.1034 - accuracy: 0.9689
Epoch 5/10
60000/60000 [==============================] - 173s 3ms/sample - loss: 0.0829 - accuracy: 0.9745
Epoch 6/10
60000/60000 [==============================] - 170s 3ms/sample - loss: 0.0682 - accuracy: 0.9792
Epoch 7/10
60000/60000 [==============================] - 171s 3ms/sample - loss: 0.0567 - accuracy: 0.9836
Epoch 8/10
60000/60000 [==============================] - 170s 3ms/sample - loss: 0.0474 - accuracy: 0.9859
Epoch 9/10
60000/60000 [==============================] - 170s 3ms/sample - loss: 0.0402 - accuracy: 0.9888
Epoch

In [20]:
longer_trajectory_hopfield_model = tf.keras.Sequential([
    tf.keras.layers.Input([28 * 28]),
    tf.keras.layers.Dense(64, use_bias=False),  # down-sampling
    HopfieldLayer(64, dt=1e-1, num_grids=50),
    tf.keras.layers.Dense(10, activation='softmax')
])

longer_trajectory_hopfield_model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'])

longer_trajectory_hopfield_model.set_weights(hopfield_model.get_weights())
longer_trajectory_hopfield_model.evaluate(x_train, y_train, batch_size=128)

60000/60000 [==============================] - 215s 4ms/sample - loss: 0.0392 - accuracy: 0.9891


[0.03918587080339591, 0.98911667]